# Variance Analysis in Multistep Bootstrapping

In order to understand the $Q(\sigma)$ algorithm better, we will analyze and compare its performance to other multistep bootstrapping methods. Since $Q(\sigma)$ is tied to the off policy n-step tree backup algorithm, we thus compare it to the off-policy variants of n-step SARSA, n-step expected SARSA, and to the n-step tree backup algorithm.

## Bootstrapping multistep methods
As [1] notes, bootstrapping methods work best when applied over multiple time steps, such that a significant and recognizable state change can be observed. From [1] also, note that multistep methods are usually associated with eligibility traces, but in this benchmark we consider only the multistep component of these algorithms.

## Theory
### Off-policy N-step SARSA and N-step expected SARSA [1]
The regular SARSA(0) algorithm learns the value of state-action pairs and, after each transition from a nonterminal state $S_t$ applies the update equation:
$$ Q(S,A) \leftarrow Q(S,A) + \alpha [ R + \gamma Q(S', A') - Q(S,A)]$$

In off-policy n-step SARSA, we still learn the value of state-action pairs but only update our estimate after $n$ steps, weighing the update using the importance sampling ratio $\rho_{t}^{t+n}$ defined as the relative probability under the two policies of taking the n actions from $A_t$ to $A_{t+n-1}$:
$$\rho_t^{t+n} = \prod_{k=t}^{min(t+n-1, T-1)}  \frac{\pi(A_k|S_k)}{\mu(A_k|S_k)}$$

And we use the update equation:

$$ Q_{t+n}(S_t,A_t) = Q_{t+n-1}(S_t,A_t) + \alpha \rho_{t+1}^{t+n} [ G_t^{(n)} - Q_{t+n-1}(S_t,A_t)] $$

with $G_t^{(n)}$ the n-step return defined in terms of estimated action values as:  

$$ G_t^{(n)} = R_{t+1} + \gamma R_{t+2} \cdots + \gamma^{n-1} R_{t+n} + \gamma^n Q_{t+n-1}(S_{t+n},a) , n \geq 1, 0\leq t \leq T-n$$


For off-policy n-step expected SARSA, we simply change the computation of the n-step return to:
$$ G_t^{(n)} = R_{t+1} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n \sum_a \pi(a|S_{t+n} Q_{t+n-1}(S_{t+n},a) , n \geq 1, 0\leq t \leq T-n$$ 

### N-step Tree Backup Algorithm [1]
In the n-step tree backup algorithm, we consider all possible actions from each state and estimate their values at step $t$. However we do not boostrap for the action that is actually taken at time $t$ as we therefore have a sample for this action. We thus go on to consider the state and actions that follow from having taken that action at time $t$, until $n$ steps have been exhausted.

Note that each of the action values of the possible actions at a specific state gets weighted by its probability of being taken, with the taken action weighing the whole tree underneath it (= all the possible actions originating from having taken that action). Therefore all the actions underneath a taken action $a$ at step $t$ will have their probabilities weighted by the probability of taking $a$ at step $t$.

We thus obtain the n-step return used in tree-backup:
$$G_t^{(n)} = Q_{t-1}(S_t, A_t) + \sum_{k=t}^{min(t+n-1, T-1)}\delta_k \prod_{i=t+1}^{k} \gamma \pi(A_i | S_i)$$

where $$\delta_t = R_{t+1} + \gamma V_{t+1} - Q_{t-1}(S_t, A_t)$$

This return is then used in the action-value update equation (same as n-step SARSA, without importance sampling): 
$$ Q_{t+n}(S_t,A_t) = Q_{t+n-1}(S_t,A_t) + \alpha [ G_t^{(n)} - Q_{t+n-1}(S_t,A_t)] $$

### N-Step $Q(\sigma)$ algorithm[1]
In the n-step $Q(\sigma)$ algorithm, we alternate between the ideas from Sarsa and the tree backup algorithm. Thus at each step, based on the value of $\sigma \in [0,1]$, the $Q(\sigma)$ update takes a fraction of the sample-based update (Sarsa) and the rest from the non sampling update (tree backup).
[1] thus generalizes the TD error $\delta_t$ to be a function of $\sigma$:
$$\delta_t = R_{t+1} + \gamma[\sigma_{t+1}Q_{t}(S_{t+1}, A_{t+1}) + (1-\sigma_{t+1}) V_{t+1}] - Q_{t-1}(S_t, A_t)$$

which is then used in the n-step return equation :
$$G_t^{(n)} = Q_{t-1}(S_t, A_t) + \sum_{k=t}^{min(t+n-1, T-1)}\delta_k \prod_{i=t+1}^{k} \gamma [(1-\sigma_i)\pi(A_i | S_i) + \sigma_i]$$

Finally we update the importance sampling ratio to take $\sigma$ into account:
$$\rho_t^{t+n} = \prod_{k=t}^{min(t+n-1, T-1)} (\sigma_k \frac{\pi(A_k|S_k)}{\mu(A_k|S_k)} + 1 - \sigma_k)$$

We can then use the above equations in the off-policy update equation from off-policy Sarsa.


## Citations and Footnotes
[1] Sutton, Richard S., and Andrew G. Barto. Reinforcement learning: An introduction. Vol. 1. No. 1. Cambridge: MIT press, 1998.
